# Example epsilon-machine implementation

Here we are considering a simple classical model: uniform renewal process. There the number of states $N$ is chosen to be $3$.

![Epsilon-machine](Renewal.png)

In [1]:
import numpy as np

from stochprocq import get_uniform_renewal
from stochprocq.Models.renewal import RenewalProcess

The uniform renewal process is provided in the `stochprocq.Models` module.

**Note**: The number of states is `len(probs) + 1`.

In [2]:
model = get_uniform_renewal(2) # 3 causal states

print(f'The probability of tick (output 1) at each memory state: {[1-p for p in model.probs] + [1]}')

print(f'The stationary distribution of the memory states are: {model.steady_state}')

The probability of tick (output 1) at each memory state: [0.33333333333333326, 0.5, 1]
The stationary distribution of the memory states are: [0.5        0.33333333 0.16666667]


To generate the stationary distribution of the outputs given a fixed length, we can do:

In [3]:
dists = model.gen_dists(3) # distribution of 3 steps outputs started from each causal state

# In the experiment we always start from the 0 state:
p_dist = dists[0]
print(p_dist)

[0.         0.33333333 0.22222222 0.11111111 0.11111111 0.11111111
 0.07407407 0.03703704]


In [4]:
# First let's see the true probabilities of reset after k steps:

# print(model.prob_seq('1', np.array([1.,0,0]))) # either specify the initial distribution
print(f'{model.prob_seq('1', past='1') = }') # or the past outputs, where '1' means the last output is 1, and the process is reset to state 0.
print(f'{model.prob_seq('01', past='1') = }')
print(f'{model.prob_seq('001', past='1') = }')

model.prob_seq('1', past='1') = np.float64(0.33333333333333326)
model.prob_seq('01', past='1') = np.float64(0.33333333333333337)
model.prob_seq('001', past='1') = np.float64(0.33333333333333337)


Now we assume in experiment we have measured the probabilities of reset after k steps starting from the 0 state:

`prob(1) = 0.37, prob(01) = 0.32, prob(001) = 0.31`

In [5]:
# Reset probabilities (emitting 1) after k steps starting from state 0 are:
q_emit = np.array([0.37, 0.32, 0.31])

# probability of emitting 0 at each causal state:
q_survive_st = np.zeros_like(q_emit)
q_survive_st[0] = q_emit[0]
for i in range(1, len(q_emit)):
    q_survive_st[i] = (q_emit[i] / np.prod(1 - q_survive_st[:i]))

print(f'Inferred probs of emitting 1 at each causal state: {q_survive_st}')


Inferred probs of emitting 1 at each causal state: [0.37       0.50793651 1.        ]


Since in the quantum model, we also hard reset the memory state to 0 after output 1, it also described by a renewal process.

In [6]:
q_model = RenewalProcess([1-q for q in q_survive_st[:-1]]) # the last state always emit 1
print(f'distribution of 3 steps outputs started from state 0:\n{q_model.gen_dists(3)[0]}')
print(f'compare with original model:\n{model.gen_dists(3)[0]}')

distribution of 3 steps outputs started from state 0:
[0.       0.31     0.2016   0.1184   0.1147   0.1184   0.086247 0.050653]
compare with original model:
[0.         0.33333333 0.22222222 0.11111111 0.11111111 0.11111111
 0.07407407 0.03703704]


This model reproduces the same statistics as the "quantum experiment":

In [7]:
print(f'{q_model.prob_seq('1', past='1') = }') # or the past outputs, where '1' means the last output is 1, and the process is reset to state 0.
print(f'{q_model.prob_seq('01', past='1') = }')
print(f'{q_model.prob_seq('001', past='1') = }')

q_model.prob_seq('1', past='1') = np.float64(0.37)
q_model.prob_seq('01', past='1') = np.float64(0.32)
q_model.prob_seq('001', past='1') = np.float64(0.31)


In [8]:
from stochprocq.measure import eval_diverge

p_dist = model.gen_dists(4)[0]
q_dist = q_model.gen_dists(4)[0]

ave_kl_div_rate = eval_diverge(p_dist, q_dist, his_steps=3)
print(f'Average KL divergence rate between the two models: {ave_kl_div_rate}')

Average KL divergence rate between the two models: 0.0025561495292130577


The minimal divergence rate achievable by a classical model with 2-dimensional memory is:

In [10]:
classical_bd = model.classical_bd(4,target_dim=2)
print(f'The minimal divergence rate achievable by a classical model with 2-dimensional memory is:\n{classical_bd}')
print(f'which is larger than the "quantum model" we artificially assigned:\n{ave_kl_div_rate}')

The minimal divergence rate achievable by a classical model with 2-dimensional memory is:
0.03145364592347788
which is larger than the "quantum model" we artificially assigned:
0.0025561495292130577
